In [1]:
import torch
from torchvision import models
import torch.nn as nn
from getData import read_data

In [2]:
# Define MLP model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(4, 10),  # 4 features to 10 hidden nodes
            nn.ReLU(),
            nn.Linear(10, 1),
            nn.Sigmoid()  # Output a single probability for binary classification
        )
    
    def forward(self, x):
        return self.layers(x)
    
# Load the models
mlp_model = MLP()
mlp_model.load_state_dict(torch.load('mlp_model.pth'))
mlp_model.eval()

image_model = models.resnet18(weights='ResNet18_Weights.DEFAULT')
num_ftrs = image_model.fc.in_features
image_model.fc = nn.Linear(num_ftrs, 2)
image_model.load_state_dict(torch.load('image_model.pth'))
image_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [3]:
image, stats = read_data()

Data has been written to CSV.
Image downloaded successfully and saved as c:\Users\jackm\Desktop\projects\fog_predictor\Images\Prediction\train\2024-05-28_15-53-24.jpg!


In [7]:
print(image)

tensor([[[-0.3994, -0.3835, -0.3675,  ..., -1.3064, -1.3701, -1.4337],
         [-0.3835, -0.3675, -0.3516,  ..., -1.2905, -1.3701, -1.4337],
         [-0.0334,  0.0462,  0.1099,  ..., -1.2905, -1.3860, -1.4178],
         ...,
         [-1.7202, -1.7679, -1.8316,  ..., -2.1658, -2.1817, -2.1976],
         [-1.9111, -1.9111, -1.9111,  ..., -2.1658, -2.2294, -2.2453],
         [-1.9589, -1.9271, -1.8793,  ..., -2.1817, -2.2453, -2.2612]],

        [[-0.2342, -0.2021, -0.2021,  ..., -1.3735, -1.4056, -1.4537],
         [-0.2342, -0.2181, -0.2021,  ..., -1.3735, -1.4056, -1.4537],
         [ 0.0707,  0.1509,  0.2312,  ..., -1.3735, -1.4216, -1.4376],
         ...,
         [-1.8228, -1.8548, -1.9511,  ..., -2.2721, -2.2881, -2.3041],
         [-1.9993, -2.0153, -2.0153,  ..., -2.2881, -2.2881, -2.3041],
         [-2.0474, -2.0153, -1.8869,  ..., -2.3041, -2.2881, -2.3041]],

        [[ 0.2153,  0.2308,  0.2463,  ..., -1.4576, -1.4576, -1.5040],
         [ 0.2308,  0.2463,  0.2463,  ..., -1

In [5]:
def predict_mlp(stats):
    # Read the most recent data entry
    data = stats
    
    # Make prediction
    with torch.no_grad():
        prediction = mlp_model(data)
    
    # Interpret the result
    probability = prediction.item()
    result = "Fog" if probability >= 0.5 else "No Fog"
    probability_percent = probability * 100  # Convert to percentage
    
    return {"probability": f"{probability_percent:.4f}%", "prediction": result}

In [6]:
predict_mlp(stats)

{'probability': '15.4355%', 'prediction': 'No Fog'}

In [23]:
tensor1 = torch.tensor([52.0000, 100.0000, 51.5000, 2.0000])
tensor2 = torch.tensor([50.0000, 100.0000, 50.0000, 2.0000])
tensor3 = torch.tensor([51.0000, 100.0000, 49.5000, 2.0000])
tensor4 = torch.tensor([51.0000, 95.0000, 49.5000, 2.0000])
#predict_mlp(tensor1)
#predict_mlp(tensor2)
#predict_mlp(tensor3)
predict_mlp(tensor4)

{'probability': '94.1582%', 'prediction': 'Fog'}

In [7]:
def predict_image(image):
    image = image.unsqueeze(0)
    with torch.no_grad():
        output = image_model(image)
    
    # Get the predicted class probabilities
    probabilities = torch.softmax(output, dim=1)
    
    # Get the predicted class (0 or 1 for binary classification)
    predicted_class = probabilities.argmax(dim=1).item()
    
    # Get the probability of the predicted class
    probability = probabilities[0, predicted_class].item()
    result = "No Fog" if predicted_class == 1 else "Fog"
    probability_percent = probability * 100  # Convert to percentage
    
    
    return {"probability": f"{probability_percent:.4f}%", "prediction": result}

In [8]:
predict_image(image)

{'probability': '98.5160%', 'prediction': 'No Fog'}